<a href="https://colab.research.google.com/github/Diegoarmando24/Tarea-3/blob/main/Ejercicio_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Función manual para descomposición de Cholesky
def cholesky_decomp(M):
    """
    Descompone la matriz simétrica positiva definida M en L @ L.T,
    donde L es triangular inferior.
    """
    n = M.shape[0]
    L = np.zeros_like(M)
    for i in range(n):
        for j in range(i + 1):
            s = np.dot(L[i, :j], L[j, :j])
            if i == j:
                # Aseguramos argumento no negativo por errores numéricos
                val = M[i, i] - s
                L[i, j] = np.sqrt(val if val > 0 else 0.0)
            else:
                L[i, j] = (M[i, j] - s) / L[j, j]
    return L

# Generación del conjunto de puntos (t_i, y_i) con perturbación aleatoria pequeña
np.random.seed(0)
n = 12
epsilon = 1e-10
t = np.linspace(0, 11, n)
x_true = np.ones(n)
A = np.vander(t, n, increasing=True)
y_true = A.dot(x_true)
y = y_true + epsilon * (2*np.random.rand(n) - 1)

# Ajuste por Ecuaciones Normales usando la descomposición de Cholesky manual
M = A.T @ A
c = A.T @ y
L = cholesky_decomp(M)               # L tal que L @ L.T ≈ M
z = np.linalg.solve(L, c)            # resuelve L z = c
x_normales = np.linalg.solve(L.T, z) # resuelve L.T x = z

# Ajuste por QR (Householder)
Q, R = np.linalg.qr(A)
x_qr = np.linalg.solve(R, Q.T @ y)

# Comparación con la solución exacta x_j = 1
err_normales = np.linalg.norm(x_normales - x_true)
err_qr      = np.linalg.norm(x_qr      - x_true)
cond_A      = np.linalg.cond(A)

print("Condición de A:", cond_A)
print("Error (Normales):", err_normales)
print("Error (QR):      ", err_qr)
print("Coeficientes (Normales):", x_normales)
print("Coeficientes (QR):      ", x_qr)

# Visualización del ajuste
tt = np.linspace(t.min(), t.max(), 300)
y_fit_norm = np.polyval(np.flip(x_normales), tt)
y_fit_qr   = np.polyval(np.flip(x_qr),       tt)

plt.scatter(t, y, color='red', label="Datos con ruido")
plt.plot(tt, y_fit_norm, label="Ajuste Normales")
plt.plot(tt, y_fit_qr, '--', label="Ajuste QR")
plt.xlabel("t")
plt.ylabel("y")
plt.title("Ajuste polinomio grado 11 con ε=1e-10")
plt.legend()
plt.show()

